In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import datetime


from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import math

In [4]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]
pd.options.display.max_columns = None


In [15]:
test = df[0:1]
test['date_time'].str.split(' ')[0]

['2012-10-02', '09:00:00']

In [39]:
datetime.datetime.strptime('2019-06-8', '%Y-%m-%d').weekday()

5

In [83]:
df = pd.read_csv('../data/Metro_Interstate_Traffic_Volume.csv', sep=',')
df[['date', 'time']] = df['date_time'].str.split(' ', expand=True)
df['hour'] = int(df['time'].str.split(':')[0][0])
df['is_weekend'] = [datetime.datetime.strptime(row['date'], '%Y-%m-%d').weekday()>=5 for id, row in df.iterrows()]
df.drop(['weather_description', 'date_time', 'date', 'time'], axis=1, inplace=True)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,traffic_volume,hour,is_weekend
0,None,288.28,0.0,0.0,40,Clouds,5545,9,False
1,None,289.36,0.0,0.0,75,Clouds,4516,9,False
2,None,289.58,0.0,0.0,90,Clouds,4767,9,False
3,None,290.13,0.0,0.0,90,Clouds,5026,9,False
4,None,291.14,0.0,0.0,75,Clouds,4918,9,False


In [85]:
df.loc[df['holiday']!='None', 'holiday'] = True
df.loc[df['holiday']=='None', 'holiday'] = False
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,traffic_volume,hour,is_weekend
0,False,288.28,0.0,0.0,40,Clouds,5545,9,False
1,False,289.36,0.0,0.0,75,Clouds,4516,9,False
2,False,289.58,0.0,0.0,90,Clouds,4767,9,False
3,False,290.13,0.0,0.0,90,Clouds,5026,9,False
4,False,291.14,0.0,0.0,75,Clouds,4918,9,False


In [86]:
df = pd.get_dummies(df, columns=['holiday', 'is_weekend', 'weather_main'])
df.head()

,temp,rain_1h,snow_1h,clouds_all,traffic_volume,hour,holiday_False,holiday_True,is_weekend_False,is_weekend_True,weather_main_Clear,weather_main_Clouds,weather_main_Drizzle,weather_main_Fog,weather_main_Haze,weather_main_Mist,weather_main_Rain,weather_main_Smoke,weather_main_Snow,weather_main_Squall,weather_main_Thunderstorm
0,288.28,0.0,0.0,40,5545,9,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1,289.36,0.0,0.0,75,4516,9,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
2,289.58,0.0,0.0,90,4767,9,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
3,290.13,0.0,0.0,90,5026,9,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
4,291.14,0.0,0.0,75,4918,9,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0


In [95]:
for col in df.columns:
    missing_count = df[df[col].isna()].shape[0]
    total_count = df.shape[0]
    print(f"{col:25} {missing_count:8} {(100*missing_count/total_count):0.2f}%")

temp                             0 0.00%
rain_1h                          0 0.00%
snow_1h                          0 0.00%
clouds_all                       0 0.00%
traffic_volume                   0 0.00%
hour                             0 0.00%
holiday_False                    0 0.00%
holiday_True                     0 0.00%
is_weekend_False                 0 0.00%
is_weekend_True                  0 0.00%
weather_main_Clear               0 0.00%
weather_main_Clouds              0 0.00%
weather_main_Drizzle             0 0.00%
weather_main_Fog                 0 0.00%
weather_main_Haze                0 0.00%
weather_main_Mist                0 0.00%
weather_main_Rain                0 0.00%
weather_main_Smoke               0 0.00%
weather_main_Snow                0 0.00%
weather_main_Squall              0 0.00%
weather_main_Thunderstorm        0 0.00%


In [52]:
def do_xgb(train_df):
    X = train_df.drop('traffic_volume', axis=1)
    y = train_df['traffic_volume']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    param = {
        'silent': 1,  # logging mode - quiet
        'objective': 'reg:squarederror',
        "eval_metric": 'rmse',
    }
    
    model = xgb.train(param, dtrain)

    train_pred = model.predict(dtrain)
    test_pred = model.predict(dtest)
    
    print(f"Train Error: {math.sqrt(mean_squared_error(y_train, train_pred))}")
    print(f"Test Error: {math.sqrt(mean_squared_error(y_test, test_pred))}")
    
    return model

In [88]:
do_xgb(df)

/home/kfarr/.local/share/virtualenvs/missing_model-GGzF5LaD/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Train Error: 1836.384171616476
Test Error: 1857.4907019990494
